This notebook is structured to read levels from an xml and perform sample calculations. We start off by installing any missing packages this notebook will need to fully run.

In [ ]:
import sys, subprocess, pkg_resources
required = {'numpy','lvlspy','matplotlib','scipy'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed
if missing:
    subprocess.check_call([sys.executable, '-m','pip','install','--quiet',*missing])


import numpy as np
import lvlspy.level as lv
import lvlspy.spcoll as lc
import lvlspy.species as ls
import lvlspy.transition as lt
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import expm_multiply

Begin by downloading an example xml file, called, appropriately enough, *example.xml*, from [OSF](https://osf.io/3f59u/).  You may instead place or upload your own file and use it by commenting out the *curl* command.

In [ ]:
!curl -o example.xml -J -L https://osf.io/w6ndg/download

Begin by creating a new species collection.

In [ ]:
new_coll = lc.SpColl()

Now ensure that the data in *example.xml* are appropriate to use with *lvlspy* by validating the XML file against the appropriate liblvls schema.

In [ ]:
new_coll.validate('example.xml')

Now read the data into the species collection by updating the (empty) collection with data from the XML file.

In [ ]:
new_coll.update_from_xml('example.xml')

Let's extract the species from the collection and set a temperature (in Kelvin)

In [ ]:
sp = new_coll.get()['my_species']
T = 1.e+9 #K

From the stored properties in the xml, energies, Einstein A coefficients, and multiplicities, with the supplied temperature, we can calculate the rate matrix

In [ ]:
rate_matrix = sp.compute_rate_matrix(T)

print('\nRate Matrix:\n')

for i in range(rate_matrix.shape[0]):
    for j in range(rate_matrix.shape[1]):
        print(i, j, rate_matrix[i, j])

The following exercises can be done by going through the steps of the other notebook without having to import an xml, but for the sake of flexing the API's power, this is the route we chose. Since we have a rate matrix, we can now evolve our species with time at a fixed temperature. The governing differential equation is $\frac{dY}{dt} = \Lambda Y$. The integrator for this system of 5 coupled equations is the Newton-Raphson method, a full description can be found at [webnucleo.org](https://webnucleo.readthedocs.io/en/latest/jupyter_notebooks.html).

The following definition calculates the vector f in the equation $A\delta = -f$ and the matrix A itself is updated with the time step. The following contains the setup and loop for the Newton-Raphson solver. If you are unfamiliar with the method, it is fully described 

In [ ]:
def f_vector(y_dt,y_i,A):
    return np.matmul(A,y_dt) - y_i

def newt_raf(y,dt,tol,rate_matrix):
    y_dt = y
    A = np.identity(len(y_dt)) - dt*rate_matrix
    delta = np.ones(len(y_dt))
    while max(delta) > tol:
        delta = np.linalg.solve(A,-f_vector(y_dt,y,A))
        y_dt = y_dt + delta

    return y_dt

The following sets up the time array, initial condition, and convergence tolerance to run the method

In [ ]:
tol =1e-6 #convergence tolerance
t_begin = 1.e-30
t_end = 100
n_steps = 200

y = np.zeros(rate_matrix.shape[0]) #initial condition. default is system sitting at level 0
y[0] = 1.0
t = np.logspace(np.log10(t_begin), np.log10(t_end), n_steps) #logarithmic space was chosen to smoothen out the graphs at small timescales

Here's the main loop that runs the Newton-Raphson method to evolve the system

In [ ]:
Y = np.empty((5,n_steps)) #set up the 2D array for graphing
Y[:,0] = y #initialize the 1st column to the initial conditions set above

for i in range(1,n_steps):
    dt = t[i] - t[i-1] #since we are using logspace for time, the timestep is not constant
    Y[:,i] = newt_raf(y,dt,tol,rate_matrix)
    y = Y[:,i]


The API can also calculate the equilibrium values beforehand. The starred values on the graph are the equilibrium values calculated straight from the API. As expected, the system evolved towards those values

In [ ]:
eq_probs = sp.compute_equilibrium_probabilities(T)

for i in range(len(sp.get_levels())):
    plt.plot(t,Y[i,:],label = str(i) + ' level')
        
plt.gca().set_prop_cycle(None)  # Reset the color cycle to align equilibria with network solutions.

for i in range(len(eq_probs)):
    plt.plot(t_end, eq_probs[i], 'x')
    
plt.xlabel('time (s)')
plt.ylabel('Probability')
plt.xlim([1e-8,1000])
plt.xscale('log')

plt.legend()
plt.show()

It is a good exercise to tweak the initial condition set above and rerun the cells to verify that the system will always evolve to the equilibrium values. Just remember to normalize the vector since these values are probabilities. For those who want a more 'built-in', you can easily use odeint or sparse solving techniques built into scipy. 

We'll start with odeint, for both we will use the same time setup as defined above

In [ ]:
def my_func(y, t, rate_matrix): #this function calculates the derivatives
    return rate_matrix.dot(y)

y = np.zeros(rate_matrix.shape[0]) #initial conditions
y[0] = 1

sol_odeint = odeint(my_func, y, t, args=(rate_matrix,))

#graph
for i in range(sol_odeint.shape[1]):
    plt.plot(t, sol_odeint[:, i], label = 'Level ' + str(i))
    
plt.gca().set_prop_cycle(None)  # Reset the color cycle to align equilibria with network solutions.

for i in range(sol_odeint.shape[1]):
    plt.plot(t[t.shape[0]-1], eq_probs[i], 'x')
    
plt.xscale('log')
plt.xlim([1.e-10, 1000])
plt.xlabel('time (s)')
plt.ylabel('Probability')
plt.legend()

And now for the sparse solver. The method built into scipy is called expm_multiply

In [ ]:
y = np.zeros(rate_matrix.shape[0]) #initial conditions
y[0] = 1

A = csc_matrix(rate_matrix) #transforming the rate matrix into its sparse form

sol_expm_solver = np.empty([t.shape[0], rate_matrix.shape[0]])
sol_expm_solver[0,:] = y
for i in range(len(t)-1):
    y = expm_multiply(A, y, start=t[i], stop=t[i+1], num=2, endpoint=True)[0,:]
    sol_expm_solver[i+1,:] = y

for i in range(sol_expm_solver.shape[1]):
    plt.plot(t, sol_expm_solver[:, i], label = 'Level ' + str(i))
    
plt.gca().set_prop_cycle(None)  # Reset the color cycle to align equilibria with network solutions.

for i in range(sol_expm_solver.shape[1]):
    plt.plot(t[t.shape[0]-1], eq_probs[i], 'x')
    
plt.xscale('log')
plt.xlim([1.e-10, 1000])
plt.xlabel('time (s)')
plt.ylabel('Probability')
plt.legend()

As you can see, the three graphs are identitcal. The fugacity evolution of each level can be calculated based on the attained solution

In [ ]:
fugacities = np.empty((Y.shape[0], Y.shape[1]))

for i in range(len(sp.get_levels())):
    fugacities[i, :] = Y[i, :] / eq_probs[i]
    
for i in range(len(sp.get_levels())):
    plt.plot(t,fugacities[i,:],label = str(i) + ' level')
    
plt.xlabel('time (s)')
plt.ylabel('Fugacity')
plt.xscale('log')
plt.xlim([1e-6,100])

As a final step to illustrate another *SpColl* method, output the data in the species collection to new files.  The first uses the default energy scale (*keV*) while the second specifically selects *eV* as the energy scale for the levels.

In [ ]:
new_coll.write_to_xml("new_example.xml")

new_coll.write_to_xml("new_example_ev.xml", units = 'eV')

!cat new_example_ev.xml